In [19]:
import numpy as np
import math

In [17]:
def tanh(x):
    return np.tanh(x)
def softmax(x_list):
#     剪掉最大值，防止指数计算的时候指数过大，但是整体的值是不变的，因为相当于在上下都除以了一个常数
    exp=np.exp(x_list-x_list.max())
    return exp/exp.sum()

In [18]:
softmax(np.array([1,2,3,4]))

array([0.0320586 , 0.08714432, 0.23688282, 0.64391426])

In [34]:
dimensions=[28*28,10]
activation=[tanh,softmax]
# 确定分布
distribution=[
    {'b':[0,0]},
    {'b':[0,0],'w':[-math.sqrt(6/(dimensions[0]+dimensions[1])),math.sqrt(6/(dimensions[0]+dimensions[1]))]}
]

NameError: name 'layer' is not defined

In [37]:
def init_parameter_b(layer):
    dist = distribution[layer]['b']
#    np.random.rand先生成0-1的随机数，然后再映射到b的分布区间中去
    return np.random.rand(dimensions[layer])*(dist[1]-dist[0])+dist[0]


def init_parameter_w(layer):
    dist = distribution[layer]['w']
#     首先是根据dimension生成一个dismensions[layer-1]*dismension[layer]的矩阵（0，1），然后将这个矩阵映射到其值域上去
# 这种方式要注意第0层是没有w的
    return np.random.rand(dimensions[layer-1],dimensions[layer])*(dist[1]-dist[0])+dist[0]


# 设计函数初始化第0层和第第一层的参数
def init_parameter():
    init_parameter_w(1)
    init_parameter_b(0)
    init_parameter_b(1)

In [38]:
init_parameter_w(1)

array([[-0.05055135, -0.07769846,  0.02399336, ..., -0.04053858,
        -0.04451544, -0.04855048],
       [ 0.05444738,  0.06181498,  0.0015491 , ..., -0.07997882,
        -0.01111511, -0.02009478],
       [ 0.03353246, -0.02048818, -0.00086384, ..., -0.07127232,
         0.06105379, -0.00501281],
       ...,
       [-0.0292312 , -0.07664309, -0.0651903 , ..., -0.07126411,
         0.02508519, -0.05807824],
       [ 0.05168886,  0.06498627,  0.07580876, ...,  0.06817693,
        -0.0392854 , -0.0800937 ],
       [ 0.01532485,  0.03502682, -0.07756618, ..., -0.07830454,
         0.05733133,  0.05321464]])